<a href="https://colab.research.google.com/github/Ruush07/ML-Brain-Anomaly-Detection/blob/main/Brain_Tumor_Detection_Using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN for **Brain** Tumor Detection Using Pytorch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import kagglehub
import os

print("📥 Downloading Dataset via KaggleHub...")
path = kagglehub.dataset_download("navoneel/brain-mri-images-for-brain-tumor-detection")
print(f"✅ Dataset downloaded to: {path}")

scan_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

data_dir = os.path.join(path, 'brain_tumor_dataset')
full_dataset = datasets.ImageFolder(root=data_dir, transform=scan_transforms)

# Split Data (80% Train / 20% Test)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_set, test_set = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

print(f"📊 Training Samples: {len(train_set)} | Testing Samples: {len(test_set)}")

AttributeError: module 'sympy' has no attribute 'printing'

In [ ]:
class BrainTumorClassifier(nn.Module):
    def __init__(self):
        super(BrainTumorClassifier, self).__init__()

        # Layer 1: Input (3 channels) -> 32 filters
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Layer 2: 32 -> 64 filters
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Layer 3: 64 -> 128 filters
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Classification Layer
        self.fc1 = nn.Linear(128 * 16 * 16, 128) # Flattened size: 128 filters * 16x16 image
        self.fc2 = nn.Linear(128, 2) # Output: Tumor (1) or Healthy (0)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5) # Prevents overfitting

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # Block 1
        x = self.pool(self.relu(self.conv2(x))) # Block 2
        x = self.pool(self.relu(self.conv3(x))) # Block 3
        x = x.view(-1, 128 * 16 * 16)           # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BrainTumorClassifier().to(device)
print(f"🚀 Model initialized on {device}")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 12 # Good balance for speed/accuracy

print("⏳ Starting Training...")
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(train_loader):.4f}")

print("✅ Training Complete!")
# Save the model
torch.save(model.state_dict(), 'brain_tumor_model.pth')
print("💾 Model saved as 'brain_tumor_model.pth'")

In [ ]:
# Visualize Results
model.eval()
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images.to(device)
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# Plotting
fig = plt.figure(figsize=(12, 8))
classes = ['Healthy', 'Tumor']

for idx in range(12):
    ax = fig.add_subplot(3, 4, idx+1, xticks=[], yticks=[])
    img = images[idx].cpu() / 2 + 0.5 # Un-normalize
    plt.imshow(img.permute(1, 2, 0))

    pred_label = classes[predicted[idx]]
    true_label = classes[labels[idx]]
    color = 'green' if pred_label == true_label else 'red'

    ax.set_title(f"Pred: {pred_label}\nTrue: {true_label}", color=color)

plt.show()